In [2]:
import logging
from enum import Enum
from abc import ABC, abstractmethod

# logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Enums and result classes
class TransactionStatus(Enum):
    PENDING = "PENDING"
    COMPLETED = "COMPLETED"
    FAILED = "FAILED"

class TransactionResult:
    def __init__(self, success: bool, transaction_id: str, message: str = ""):
        self.success = success
        self.transaction_id = transaction_id
        self.message = message

class NetworkException(Exception):
    pass

class PaymentException(Exception):
    pass

class RefundException(Exception):
    pass

# Abstract class for PaymentGateway
class PaymentGateway(ABC):
    @abstractmethod
    def charge(self, user_id: str, amount: float) -> TransactionResult:
        pass

    @abstractmethod
    def refund(self, transaction_id: str) -> TransactionResult:
        pass

    @abstractmethod
    def get_status(self, transaction_id: str) -> TransactionStatus:
        pass

# PaymentProcessor implementation
class PaymentProcessor:
    def __init__(self, payment_gateway: PaymentGateway):
        self.payment_gateway = payment_gateway

    def process_payment(self, user_id: str, amount: float) -> TransactionResult:
        if not isinstance(amount, (int, float)) or amount <= 0:
            raise ValueError("Amount must be a positive number")
        if not user_id or not user_id.strip():
            raise ValueError("User ID cannot be empty or whitespace")

        logger.info("Processing payment for user %s with amount %.2f", user_id, amount)
        try:
            result = self.payment_gateway.charge(user_id, amount)
            logger.info("Payment successful: %s", result.transaction_id)
            return result
        except (NetworkException, PaymentException) as e:
            logger.error("Payment failed for user %s: %s", user_id, str(e))
            return TransactionResult(success=False, transaction_id="", message=str(e))

    def refund_payment(self, transaction_id: str) -> TransactionResult:
        if not transaction_id or not transaction_id.strip():
            raise ValueError("Transaction ID cannot be empty or whitespace")

        logger.info("Processing refund for transaction %s", transaction_id)
        try:
            result = self.payment_gateway.refund(transaction_id)
            logger.info("Refund successful: %s", transaction_id)
            return result
        except (NetworkException, RefundException) as e:
            logger.error("Refund failed for transaction %s: %s", transaction_id, str(e))
            return TransactionResult(success=False, transaction_id="", message=str(e))

    def get_payment_status(self, transaction_id: str) -> TransactionStatus:
        if not transaction_id or not transaction_id.strip():
            raise ValueError("Transaction ID cannot be empty or whitespace")

        logger.info("Checking status for transaction %s", transaction_id)
        try:
            status = self.payment_gateway.get_status(transaction_id)
            logger.info("Transaction %s status: %s", transaction_id, status.value)
            return status
        except NetworkException as e:
            logger.error("Failed to retrieve status for transaction %s: %s", transaction_id, str(e))
            return TransactionStatus.FAILED


In [7]:
import unittest
from unittest.mock import MagicMock
from unittest.mock import call

class TestPaymentProcessor(unittest.TestCase):
    def setUp(self):
        self.mock_gateway = MagicMock()
        self.processor = PaymentProcessor(self.mock_gateway)

    def test_process_payment_success(self):
        self.mock_gateway.charge.return_value = TransactionResult(True, "txn_12345", "Success")
        result = self.processor.process_payment("user_1", 100.0)
        self.mock_gateway.charge.assert_called_once_with("user_1", 100.0)
        self.assertTrue(result.success)
        self.assertEqual(result.transaction_id, "txn_12345")

    def test_process_payment_invalid_amount(self):
        with self.assertRaises(ValueError):
            self.processor.process_payment("user_1", -50.0)

    def test_process_payment_empty_user_id(self):
        with self.assertRaises(ValueError):
            self.processor.process_payment("", 100.0)

    def test_process_payment_network_exception(self):
        self.mock_gateway.charge.side_effect = NetworkException("Network issue")
        result = self.processor.process_payment("user_1", 100.0)
        self.assertFalse(result.success)
        self.assertEqual(result.message, "Network issue")

    def test_refund_payment_success(self):
        self.mock_gateway.refund.return_value = TransactionResult(True, "txn_12345", "Refunded")
        result = self.processor.refund_payment("txn_12345")
        self.mock_gateway.refund.assert_called_once_with("txn_12345")
        self.assertTrue(result.success)
        self.assertEqual(result.message, "Refunded")

    def test_refund_payment_invalid_transaction_id(self):
        with self.assertRaises(ValueError):
            self.processor.refund_payment("")

    def test_refund_payment_exception(self):
        self.mock_gateway.refund.side_effect = RefundException("Refund error")
        result = self.processor.refund_payment("txn_12345")
        self.assertFalse(result.success)
        self.assertEqual(result.message, "Refund error")

    def test_get_payment_status_completed(self):
        self.mock_gateway.get_status.return_value = TransactionStatus.COMPLETED
        status = self.processor.get_payment_status("txn_12345")
        self.mock_gateway.get_status.assert_called_once_with("txn_12345")
        self.assertEqual(status, TransactionStatus.COMPLETED)

    def test_get_payment_status_invalid_transaction_id(self):
        with self.assertRaises(ValueError):
            self.processor.get_payment_status("")

    def test_get_payment_status_network_exception(self):
        self.mock_gateway.get_status.side_effect = NetworkException("Network issue")
        status = self.processor.get_payment_status("txn_12345")
        self.assertEqual(status, TransactionStatus.FAILED)

if __name__ == "__main__":
    import unittest
    unittest.main(argv=['first-arg-is-ignored'], exit=False)



..ERROR:__main__:Failed to retrieve status for transaction txn_12345: Network issue
...ERROR:__main__:Payment failed for user user_1: Network issue
..ERROR:__main__:Refund failed for transaction txn_12345: Refund error
...
----------------------------------------------------------------------
Ran 10 tests in 0.036s

OK
